In [3]:
import os 

In [1]:
%pwd

'c:\\Users\\OMEN\\Malaria_Project\\notebook\\data'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\OMEN\\Malaria_Project\\notebook'

In [7]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\OMEN\\Malaria_Project'

In [10]:
from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen=True)
class DataingestionConfig:
    root_dir : Path
    source_url : str 
    local_data_file : Path
    unzip_dir : Path

In [24]:
from src.malaria_parasite_project.constants import *
from src.malaria_parasite_project.utils.common import read_yaml, create_directories

In [25]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config= read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_roots])



    def get_data_ingestion_config(self) -> DataingestionConfig:
        config = self.config.data_ingestion 

        create_directories([config.root_dir])

        data_ingestion_config = DataingestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )  

        return data_ingestion_config


In [26]:
import os
import zipfile
import urllib.request
from pathlib import Path
from src.logger import logging
from src.malaria_parasite_project.utils.common import get_size


In [27]:
class Dataingestion: 
    def __init__(self, config: DataingestionConfig):
        self.config = config


    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urllib.request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logging.info(f"{filename}Download! with following info:\n{headers}")
        else:
            logging.info(f"File already exists of size:  {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        print(f"Extracted zip file to {unzip_path}")
       



In [29]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = Dataingestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

YAML file: config\config.yaml loaded successfully
YAML file: params.yaml loaded successfully


BoxKeyError: "'ConfigBox' object has no attribute 'artifacts_roots'"